In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from   sklearn.linear_model import LinearRegression, LogisticRegression
from   sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB

In [4]:
shark = pd.read_excel("shark_attack_data_cleaned.xlsx", nrows = 1198)

C:\Users\pedro\anaconda3\Lib\site-packages\openpyxl\worksheet\_reader.py:211: UserWarning: Cell A1048576 is marked as a date but the serial value 14707374 is outside the limits for dates. The cell will be treated as an error.
  warn(msg)


In [6]:
shark.sample(10)

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Fatal,Time,Species
530,2002-12-16 00:00:00,2002,Unprovoked,AUSTRALIA,Queensland,Miami Lake,Swimming/Diving,M,23,1,2.30.00 AM,Bull
783,1991-02-12 00:00:00,1991,Unprovoked,SOUTH AFRICA,Western Cape Province,Miller’s Point,Fishing,M,20,0,12.30.00 PM,White
461,2004-10-21 00:00:00,2004,Unprovoked,AUSTRALIA,New South Wales,Stockton Beach,Board involved,M,59,0,Afternoon,Bronze whaler
954,1974-09-02 00:00:00,1974,Unprovoked,USA,California,"Franklin Point, San Mateo County",Swimming/Diving,M,48,0,5.30.00 PM,White
870,1983-07-12 00:00:00,1983,Unprovoked,USA,Florida,"Caloosahatchee River, Lee County",Board involved,F,39,0,Afternoon,Hammerhead
454,2005-08-24 00:00:00,2005,Unprovoked,USA,California,"Scripps, LaJolla, San Diego County",Board involved,M,37,0,11.00.00 AM,White
546,2002-05-31 00:00:00,2002,Unprovoked,USA,Florida,"St. George Island (near Apachicola), Franklin ...",Swimming/Diving,M,16,0,4.15.00 PM,Unidentified species
936,1975-06-23 00:00:00,1975,Unprovoked,USA,South Carolina,"Windy hill, near Myrtle Beach, horry County",Swimming/Diving,M,18,0,4.30.00 PM,Unidentified species
148,2013-02-10 00:00:00,2013,Unprovoked,USA,Florida,"""Stuart Rocks"", Martin County",Board involved,M,16,0,2.00.00 PM,Blacktip
566,2002-11-28 00:00:00,2002,Unprovoked,USA,California,"Salmon Creek, Sonoma County",Board involved,M,48,0,9.45.00 AM,White


In [12]:
columns = {} # creating a mapper dictionary to change the names of my columns from uppercase to lowercase, this is just for my ease of use!
for column in shark.columns:
    columns[column] = column.lower()

In [15]:
shark.rename(columns = columns, inplace = True)

In [17]:
shark.info() # printing out general info of our dataframe to see which columns need work

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      1198 non-null   object
 1   year      1198 non-null   int64 
 2   type      1198 non-null   object
 3   country   1198 non-null   object
 4   area      1198 non-null   object
 5   location  1198 non-null   object
 6   activity  1198 non-null   object
 7   sex       1198 non-null   object
 8   age       1198 non-null   object
 9   fatal     1198 non-null   int64 
 10  time      1192 non-null   object
 11  species   1198 non-null   object
dtypes: int64(2), object(10)
memory usage: 112.4+ KB


In [19]:
shark.year.describe() #year column seems to be all integer values which is great for now

count    1198.000000
mean     1991.931553
std        23.287287
min      1880.000000
25%      1981.000000
50%      2000.000000
75%      2009.000000
max      2018.000000
Name: year, dtype: float64

In [9]:
shark.type.unique() # from here we can see that there are four values in type, each associated with the type of attack

array(['Unprovoked', 'Provoked', 'Boating', 'Invalid', 'Sea Disaster'],
      dtype=object)

In [23]:
shark.type.value_counts() # here we can see that Unprovoked is the most common kind of attack

Unprovoked      1087
Provoked          89
Boating           15
Sea Disaster       4
Invalid            3
Name: type, dtype: int64

In [21]:
shark.loc[shark.type == "Invalid"] # since there are only three values I will fill in these values with the most frequent value of the four categorical variables

,date,year,type,country,area,location,activity,sex,age,fatal,time,species
109,2015-06-24 00:00:00,2015,Invalid,AUSTRALIA,Western Australia,Denmark,Board involved,F,37,0,9.30.00 AM,White
308,2009-09-12 00:00:00,2009,Invalid,USA,North Carolina,"Corolla, Currituck County",Swimming/Diving,M,60,1,9.00.00 PM,Unidentified species
593,2001-09-16 00:00:00,2001,Invalid,USA,Florida,"2 miles off Pompano Beach, Broward County",Swimming/Diving,M,42,1,1.20.00 PM,Unidentified species


In [28]:
shark.type[[109,308,593]] = "Unprovoked" # updating the values of these 3 entries

C:\Users\pedro\AppData\Local\Temp\ipykernel_14464\2617648377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shark.type[[109,308,593]] = "Unprovoked"


In [30]:
shark.type.value_counts() # checking our new value counts for our updated values

Unprovoked      1090
Provoked          89
Boating           15
Sea Disaster       4
Name: type, dtype: int64

In [32]:
shark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198 entries, 0 to 1197
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   date      1198 non-null   object
 1   year      1198 non-null   int64 
 2   type      1198 non-null   object
 3   country   1198 non-null   object
 4   area      1198 non-null   object
 5   location  1198 non-null   object
 6   activity  1198 non-null   object
 7   sex       1198 non-null   object
 8   age       1198 non-null   object
 9   fatal     1198 non-null   int64 
 10  time      1192 non-null   object
 11  species   1198 non-null   object
dtypes: int64(2), object(10)
memory usage: 112.4+ KB


In [35]:
shark["time"].value_counts()

Afternoon      59
11.00.00 AM    55
4.30.00 PM     43
4.00.00 PM     42
Morning        42
               ..
6.50.00 PM      1
1.55.00 PM      1
9.00.00 PM      1
15j45           1
                1
Name: time, Length: 185, dtype: int64

### A lot of work needs to be done to clean up this time column to be ready to convert use a one hot encoder

In [ ]:
time_list = ["Early morning", "Just before sundown", "Before 07.00", "Midday", "Early afternoon", "Morning", "After noon", "4/9/1902  12.00.00 AM", "Afternoon", "Morning", "Night", "Evening"]

In [51]:
from datetime import datetime
# defining a function to convert am and pm time to 24 hour time
def strip_am_pm(time_string):
    in_time = datetime.strptime(time_string, "%I:%M %p")
    out_time = datetime.strftime(in_time, "%H:%M")
    return(out_time)

In [52]:
# shark["time"].apply(lambda time: strip_am_pm(time))

In [53]:
shark.head()

,date,year,type,country,area,location,activity,sex,age,fatal,time,species
0,2018-05-26 00:00:00,2018,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Swimming/Diving,M,15,0,5.00 PM,Bull
1,2018-02-15 00:00:00,2018,Unprovoked,AUSTRALIA,New South Wales,"Surf Beach, Kiama",Swimming/Diving,M,35,0,4.00.00 PM,Bull
2,2018-05-24 00:00:00,2018,Provoked,AUSTRALIA,Queensland,Cairns Aquarium,Shark involved,M,32,0,Morning,Grey reef
3,2018-03-31 00:00:00,2018,Unprovoked,USA,hawaii,Kukio Beach,Board involved,M,25,0,9.30.00 AM,Tiger
4,2018-03-09 00:00:00,2018,Unprovoked,AUSTRALIA,Victoria,Winkipop,Board involved,M,18,0,7.30.00 AM,Unidentified species


In [54]:
shark.columns

Index(['date', 'year', 'type', 'country', 'area', 'location', 'activity',
       'sex', 'age', 'fatal', 'time', 'species'],
      dtype='object')

In [116]:
shark_clean = shark[['type', 'country', 'activity',
       'sex', 'fatal', 'species']]

In [117]:
X = shark_clean.drop(columns = ["fatal"])
y = shark_clean["fatal"]

In [137]:
from sklearn.preprocessing import OneHotEncoder

OHE = OneHotEncoder(drop = "first")

X_encoded = OHE.fit_transform(X)

In [145]:
shark_encoded_age = pd.DataFrame(X_encoded.todense()).join(shark.age)

In [151]:
model = LogisticRegression()

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train) # tell it known stuff

y_predict = model.predict(X_test) # given future input, predict future output!

In [156]:
from sklearn.metrics import classification_report

classification_report(y_test, y_predict, output_dict = True)[""]

{'precision': 0.8864628820960698,
 'recall': 0.9759615384615384,
 'f1-score': 0.9290617848970253,
 'support': 208}